In [198]:
import os
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"

In [199]:
import numpy as np
import pandas as pd
import scipy as sp
import mne
import keras
import matplotlib.pyplot as plt
import tensorflow as tf

from keras.models import Sequential
from keras.layers import LSTM,Dense ,GRU,SimpleRNN,Conv1D,MaxPooling1D,Flatten , Dropout, BatchNormalization
from keras.optimizers import Adam
from sklearn.model_selection import train_test_split

tf.compat.v1.disable_eager_execution()

from torch import optim
import argparse
import os
import pickle
import shutil
from random import random
from torch.utils.data import DataLoader, Dataset
from torch.nn.utils.rnn import pad_sequence, pack_padded_sequence, pad_packed_sequence
import torch
import torch.nn as nn
import torch.nn.functional as F

import h5py

In [200]:
DEVICE = torch.device('cuda:0')

In [201]:
def prepad0(data, seglength):
    pad = (-len(data)) % seglength
    l = np.concatenate((data,np.zeros((pad, len(data[0])))))
    return l

In [202]:
def pad(rawdata, seglength):
    pad_length = -rawdata.size()[0] % seglength
    padded = torch.cat((rawdata, torch.zeros((pad_length, rawdata.size()[1]))), 0)
    return padded

In [203]:
def to_epochs(rawdata, seg_length = 9000):
    data = pad(rawdata, seg_length)
    return torch.split(data, seg_length)

In [204]:
def to_gpu(x, on_cpu=False, gpu_id=None):

    if torch.cuda.is_available() and not on_cpu:
        x = x.cuda(gpu_id)
    return x

In [205]:
rawdata = torch.Tensor(pd.read_csv(r"C:\Users\admin\Desktop\test excel\LB.csv", skiprows=2, header=None).values)

In [206]:
seg_length = 400
chunks = to_epochs(rawdata, seg_length)


In [208]:
class GRU(nn.Module):
    def __init__(self, input_size, output_size, hidden_dim, n_layers):
        super(GRU, self).__init__()
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers
        self.gru = nn.GRU(input_size, hidden_dim, n_layers, batch_first=True, dropout=0.2)
        self.fc = nn.Linear(hidden_dim, output_size).float()
        self.relu = nn.ReLU()

    def forward(self, x):
        out, h = self.gru(x)
        out = self.fc(self.relu(out))
        return out, h
    
    def init_hidden(self, batch_size):
        weight = next(self.parameters()).data
        hidden = weight.new(self.n_layers, batch_size, self.hidden_dim).zero_()
        return hidden